# How to connect to Azure OpenAI service via PromptSail and Langchain SDK

First import the OpenAI Python SDK and load your API key from the environment.


**You will have create Azure OpenAI deployment and get the API key from the Azure portal.**


In [1]:
import os
from dotenv import load_dotenv, dotenv_values
from pprint import pprint


config = dotenv_values(".env")


azure_oai_key = config["AZURE_OPENAI_API_KEY"]
api_base_url = config["AZURE_OPENAI_ENDPOINT"]

deployment_name = config["AZURE_OPENAI_DEPLOYMENT_NAME"]
api_version = config["AZURE_OPENAI_API_VERSION"]

print(
    f"Azure OpenAI api key={azure_oai_key[0:3]}...{azure_oai_key[-5:]}"
)
print(
    f"Azure OpenAI api endpoint={api_base_url[0:17]}..."
)
print(f"Azure OpenAI deployment name={deployment_name[0:7]}...")
print(f"Azure OpenAI api version={api_version}")

Azure OpenAI api key=9b2...d67b3
Azure OpenAI api endpoint=https://openai-pr...
Azure OpenAI deployment name=gpt-35T...
Azure OpenAI api version=2023-07-01-preview


Test the direct connection to Azure OpenAI and Langchain SDK.


In [2]:

from langchain.chat_models import AzureChatOpenAI
#from langchain_community.chat_models import AzureChatOpenAI

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage


messages = [
    SystemMessage(
        content="You are a helpful assistant that help rewirte an jira ticket."
    ),
    HumanMessage(
        content="Give meaningful title to this bug, RuntimeError: CUDA out of memory. Tried to allocate X MiB (GPU X; X GiB total capacity; X GiB already allocated; X MiB free; X cached)"
    ),
]


In [3]:

chat = AzureChatOpenAI(
    openai_api_key=azure_oai_key,
    deployment_name=deployment_name,
    api_version=api_version,
    azure_endpoint=api_base_url,
    #base_url=api_base_url,
    #model=deployment_name,
)

In [4]:
chat(messages)

AIMessage(content='Bug: RuntimeError: Insufficient CUDA Memory Allocation (GPU X; X GiB total capacity; X GiB already allocated; X MiB free; X cached)')

## Create a request to the AzureOpenAI via PromptSail proxy

Run the docker and go to PromptSail UI http://localhost/

Create new project with you `project_slug`or edit existing one.

Add your own Azure OpenAI provider by editing the project settings, this will map the Azure OpenAI endpoint to new proxy prompt sail URL. 

Set the `api base url` to your Azure OpenAI endpoint like
 
'https://**azure_openai_resource**.openai.azure.com/'
 
 and add meaningfull `deployment name`.


In mongo it will create new entry in `ai_providers` array, similar to this one

```bash
{
     ai_providers: [
        {
            deployment_name: 'azure US deployment',
            slug: 'azure-us-deployment',
            api_base: 'https://[azure_openai_resource].openai.azure.com/',
            description: '',
            provider_name: 'Azure OpenAI'
        }
    ],
}
```

In this case we will use the default `project 2` settings:
* with project_slug -> 'project2' 
* deployment_name -> 'azure-us-deployment'
resulting in promptsail proxy url like this: 

**"http://localhost:8000/project2/azure-us-deployment"**



In [5]:
ps_api_base = "http://localhost:8000/project2/azure-us-deployment"

ps_api_base = "http://localhost:8000/edu-project/ps-us2"


chat = AzureChatOpenAI(
    openai_api_key=azure_oai_key,
    deployment_name=deployment_name,
    api_version=api_version,
    azure_endpoint=ps_api_base,

)

In [6]:
chat(messages)

AIMessage(content='Bug: CUDA Out of Memory Error during Memory Allocation\n\nDescription:\nA runtime error is encountered when attempting to allocate memory in the CUDA framework. The error message states that X MiB of memory was attempted to be allocated, but the GPU X has reached its capacity. Of the X GiB total capacity, X GiB has already been allocated, leaving X MiB free. Additionally, there is X MiB of memory cached.\n\nSteps to Reproduce:\n1. Perform actions that require GPU memory allocation.\n2. Observe the CUDA Out of Memory error.\n\nExpected Result:\nMemory allocation should be successful without encountering any runtime errors.\n\nActual Result:\nA RuntimeError occurs, indicating that the GPU has run out of memory during the memory allocation process.\n\nAdditional Information:\n- GPU Details: [Specify GPU details here]\n- Total GPU Capacity: [Specify total GPU capacity here]\n- Memory Already Allocated: [Specify amount of memory already allocated here]\n- Free Memory: [Sp